In [1]:
#|default_exp toolloop

# Tool Loop

The code for Claudette's tool loop should essentially work as is. We'll replicate the whole original notebook just to make sure

In [1]:
#| export

from gaspare.core import *
from fastcore.utils import *
from fastcore.meta import delegates

from google import genai

In [2]:
model = models[2]
model

'gemini-2.0-flash'

Let's use the [same example](https://github.com/anthropics/anthropic-cookbook/blob/main/tool_use/customer_service_agent.ipynb) from Claudette's documentation.

In [3]:
orders = {
    "O1": dict(id="O1", product="Widget A", quantity=2, price=19.99, status="Shipped"),
    "O2": dict(id="O2", product="Gadget B", quantity=1, price=49.99, status="Processing"),
    "O3": dict(id="O3", product="Gadget B", quantity=2, price=49.99, status="Shipped")}

customers = {
    "C1": dict(name="John Doe", email="john@example.com", phone="123-456-7890",
               orders=[orders['O1'], orders['O2']]),
    "C2": dict(name="Jane Smith", email="jane@example.com", phone="987-654-3210",
               orders=[orders['O3']])
}

As with Claudette, we do not have to create the JSON schema manually. We can use docments.

In [4]:
def get_customer_info(
    customer_id:str # ID of the customer
): # Customer's name, email, phone number, and list of orders
    "Retrieves a customer's information and their orders based on the customer ID"
    print(f'- Retrieving customer {customer_id}')
    return customers.get(customer_id, "Customer not found")

def get_order_details(
    order_id:str # ID of the order
): # Order's ID, product name, quantity, price, and order status
    "Retrieves the details of a specific order based on the order ID"
    print(f'- Retrieving order {order_id}')
    return orders.get(order_id, "Order not found")

def cancel_order(
    order_id:str # ID of the order to cancel
)->bool: # True if the cancellation is successful
    "Cancels an order based on the provided order ID"
    print(f'- Cancelling order {order_id}')
    if order_id not in orders: return False
    orders[order_id]['status'] = 'Cancelled'
    return True

We are ready to go. The main difference here is that we don't assign the tools to the chat itself, since otherwise Gemini becomes too eager to use them.

In [5]:
tools = [get_customer_info, get_order_details, cancel_order]
chat = Chat(model)

In [6]:
r = chat('Can you tell me the email address for customer C1?', tools=tools, use_afc=False)
r

- Retrieving customer C1


<ul><li><code>get_customer_info(customer_id=C1)</code></li></ul>
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_call</code>: <ul><li><code>name</code>: get_customer_info</li><li><code>args</code>: <ul><li><b>customer_id</b>: C1</li></ul></li></ul></li></ul></details></li></ul></li><li><code>avg_logprobs</code>: -0.00010187082225456833</li><li><code>finish_reason</code>: FinishReason.STOP</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 147; Out: 10; Total: 157</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [7]:
chat()

The email address for customer C1 is john@example.com.<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: The email address for customer C1 is john@example.com.
</li></ul></details></li></ul></li><li><code>avg_logprobs</code>: -2.561106230132282e-05</li><li><code>finish_reason</code>: FinishReason.STOP</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 71; Out: 15; Total: 86</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [8]:
sp = """You will be provided with tools, but don't limit your answer to those tools.
If the user query is related to some of the tools you have access to come up with a sequence of actions to achieve the goal and **execute the plan immediately**.

If the user query is unrelated to the tools you have access to, answer the query using your own knowledge."""

chat = Chat(model)
r = chat('Cancel all orders for customer C1.', tools=tools, use_afc=False, sp=sp, temp=0.)
r

- Retrieving customer C1


I can only cancel orders if I have the order ID. I need to get the order IDs for customer C1 first.  I will use the `get_customer_info` to retrieve the customer's information and their orders, then iterate through the orders and cancel them.<br /><br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: I can only cancel orders if I have the order ID. I need to get the order IDs for customer C1 first.  I will use the `get_customer_info` to retrieve the customer's information and their orders, then iterate through the orders and cancel them.

</li></ul></details>
<details open='true'><summary>parts[1]</summary><ul><li><code>function_call</code>: <ul><li><code>name</code>: get_customer_info</li><li><code>args</code>: <ul><li><b>customer_id</b>: C1</li></ul></li></ul></li></ul></details></li></ul></li><li><code>avg_logprobs</code>: -0.07763530927545884</li><li><code>finish_reason</code>: FinishReason.STOP</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 218; Out: 68; Total: 286</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [9]:
r.function_calls

[FunctionCall(id=None, args={'customer_id': 'C1'}, name='get_customer_info')]

In [10]:
#| exports

@patch
@delegates(genai.chats.Chat.__call__)
def toolloop(self:genai.chats.Chat,
             pr, # Prompt to pass to Gemini
             max_steps=10, # Maximum number of tool requests to loop through
             trace_func:Optional[callable]=None, # Function to trace tool use steps (e.g `print`)
             cont_func:Optional[callable]=noop, # Function that stops loop if returns False
             **kwargs):
    "Add prompt `pr` to dialog and get a response from Gemini, automatically following up with `tool_use` messages"
    n_msgs = len(self.h)
    kwargs["use_afc"] = False
    r = self(pr, **kwargs)
    for i in range(max_steps):
        if not r.function_calls:break
        if trace_func: trace_func(self.h[n_msgs:]); n_msgs = len(self.h)
        r = self(**kwargs)
        if not (cont_func or noop)(self.h[-2]): break
    if trace_func: trace_func(self.h[n_msgs:])
    return r

In [11]:
chat = Chat(model)
r = chat.toolloop('Tell me the email address for customer C1.', tools=tools, sp=sp, temp=0.)
r

- Retrieving customer C1


The email address for customer C1 is john@example.com.<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: The email address for customer C1 is john@example.com.
</li></ul></details></li></ul></li><li><code>avg_logprobs</code>: -0.0002421900009115537</li><li><code>finish_reason</code>: FinishReason.STOP</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 279; Out: 15; Total: 294</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [12]:
def print_msgs(msgs):
    for n, m in enumerate(msgs):
        for i, part in enumerate(m.parts):
            print(f"\nMessage {n+1}, Part {i + 1}:\n")
            c = "* Text *: " + part.text if part.text  else "" 
            c += "* Function Call *: " + str(part.function_call) if part.function_call else ""
            c += "* Function Response *: " + str(part.function_response.response['result']) if part.function_response else ""
            print(c)
            print()

chat = Chat(model)
r = chat.toolloop('Cancel all orders for customer C1.', tools=tools, trace_func=print_msgs, temp=0., sp=sp)
r

- Retrieving customer C1

Message 1, Part 1:

* Text *: Cancel all orders for customer C1.


Message 2, Part 1:

* Text *: I can only cancel orders if I have the order ID. I need to retrieve the customer's information first to identify their orders. After that, I can cancel each order individually.




Message 2, Part 2:

* Function Call *: id=None args={'customer_id': 'C1'} name='get_customer_info'

- Cancelling order O1
- Cancelling order O2

Message 1, Part 1:

* Function Response *: {'name': 'John Doe', 'email': 'john@example.com', 'phone': '123-456-7890', 'orders': [{'id': 'O1', 'product': 'Widget A', 'quantity': 2, 'price': 19.99, 'status': 'Cancelled'}, {'id': 'O2', 'product': 'Gadget B', 'quantity': 1, 'price': 49.99, 'status': 'Cancelled'}]}


Message 2, Part 1:

* Text *: I have retrieved the customer's information. John Doe has two orders: O1 and O2. I will now cancel each order.



Message 2, Part 2:

* Function Call *: id=None args={'order_id': 'O1'} name='cancel_order'




Both orders O1 and O2 have been cancelled successfully.<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: Both orders O1 and O2 have been cancelled successfully.
</li></ul></details></li></ul></li><li><code>avg_logprobs</code>: -0.16903127156771147</li><li><code>finish_reason</code>: FinishReason.STOP</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 368; Out: 13; Total: 381</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [13]:
chat.toolloop('What is the status of order O1?', tools=tools)

- Retrieving order O1


The status of order O1 is Cancelled.<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: The status of order O1 is Cancelled.
</li></ul></details></li></ul></li><li><code>avg_logprobs</code>: -0.02214123457670212</li><li><code>finish_reason</code>: FinishReason.STOP</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 341; Out: 10; Total: 351</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

## Code interpreter

In [14]:
from toolslm.shell import get_shell
from fastcore.meta import delegates
import traceback

In [113]:
imps = 'os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses'

def CodeChat(model: Optional[str] = None, ask:bool=True, tools=None, **kwargs):
    imps = 'os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses'
    chat = Chat(model=model, **kwargs)
    chat.ask = ask
    chat.shell = get_shell()
    chat.shell.run_cell('import '+ imps)
    chat._tools = tools
    chat._tools.append(chat.run_code)
    return chat

In [206]:
@patch
def run_code(
    self:genai.chats.Chat,
    code:str,   # Code to execute in persistent IPython session
): # Result of expression on last line (if exists); '#DECLINED#' if user declines request to execute
    "Executes python code using a persistent IPython session. It asks the user for permission before executing the code."
    confirm = f'Press Enter to execute, or enter "n" to skip?\n```\n{code}\n```\n'
    if input(confirm): return '#DECLINED#'
    try: res = self.shell.run_cell(code)
    except Exception as e: return traceback.format_exc()
    return res.stdout if res.result is None else res.result

In [186]:
sp = f'''You are a knowledgable coding assistant assistant. 
Don't do complex calculations yourself -- create code for them.
You always have the ability to execute code with the `run_code` tool at your disposal.
The following modules are pre-imported for `run_code` automatically:

{imps}

Note that `run_code` interpreter state is *persistent* across calls. 

If a tool returns `#DECLINED#` report to the user that the attempt was declined and no further progress can be made.'''

In [209]:
def get_user(ignored:str # Unused parameter
            ): # Username of current user
    """Get the username of the user running this session
    
    Example: get_user()"""
    print("Looking up username")
    return 'Miko'

In [210]:
chat = CodeChat(model, tools=[get_user], sp=sp, ask=True, temp=0.6)

In [211]:
chat("What is already imported in the IPython environment?", tools=chat._tools)

I can list the libraries that are already imported in the IPython environment for you. These are:<br /><br />os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: I can list the libraries that are already imported in the IPython environment for you. These are:

os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses
</li></ul></details></li></ul></li><li><code>avg_logprobs</code>: -0.02876930996991586</li><li><code>finish_reason</code>: FinishReason.STOP</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 198; Out: 69; Total: 267</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [212]:
pr = '''Create a 1-line function `checksum` for a string `s`, that multiplies together the ascii 
values of each character in `s` using `reduce`.'''
r = chat.toolloop(pr, trace_func=print_msgs, tools=chat._tools)
r



Message 1, Part 1:

* Text *: Create a 1-line function `checksum` for a string `s`, that multiplies together the ascii 
values of each character in `s` using `reduce`.


Message 2, Part 1:

* Text *: ```python
def checksum(s):
  from functools import reduce
  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)
```




```python<br />def checksum(s):<br />  from functools import reduce<br />  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)<br />```<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: ```python
def checksum(s):
  from functools import reduce
  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)
```
</li></ul></details></li></ul></li><li><code>avg_logprobs</code>: -0.034469225189902565</li><li><code>finish_reason</code>: FinishReason.STOP</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 306; Out: 44; Total: 350</li><li><code>automatic_function_calling_history</code>: </li></ul></details>

In [213]:
_ = chat.toolloop('Run the code in the IPython environment', tools=chat._tools, sp=sp, temp=0.6, trace_func=print_msgs)

In [214]:
chat.client._genconf??

Signature: chat.client._genconf(**kw)
Source:   
@patch
def _genconf(self: genai.models.Models, **kw):
    """Builds a GenerateContentConfigDict from call parameters"""
    config= {k: v for k, v in kw.items() if k in types.GenerateContentConfigDict.__annotations__}
    if _sp := kw.get("sp", False) or kw.get('system_instruction', False) or getattr(self, 'sp', False):
        config['system_instruction'] = _sp 
    if _temp := kw.get("temp", False) or kw.get('temperature', False) or getattr(self, 'temp', False):
        config['temperature'] = _temp
    if maxtok := kw.get("maxtok", False): config['max_output_tokens'] = maxtok
    if stop := kw.get("stop", False): config['stop_sequences'] = [stop] if isinstance(stop, str) else stop

    if tools:= kw.get("tools", False):
        self._tools = tools
        config['tools'] = prep_tools(tools, toolify_everything=not kw.get("use_afc", True))
        tc = config.get('tool_config', dict())
        fcc = tc.get('function_calling_config', dic

In [215]:
chat.send_message('Run the code in the IPython environment', config={'tools': chat._tools})

Press Enter to execute, or enter "n" to skip?
```

def checksum(s):
  from functools import reduce
  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)

```
 


OK. I have run the code in the IPython environment. The function `checksum` is now defined.<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: OK. I have run the code in the IPython environment. The function `checksum` is now defined.
</li></ul></details></li></ul></li><li><code>avg_logprobs</code>: -0.01952469737633415</li><li><code>finish_reason</code>: FinishReason.STOP</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 264; Out: 23; Total: 287</li><li><code>automatic_function_calling_history</code>: <details open='true'><summary>automatic_function_calling_history[0]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: What is already imported in the IPython environment?</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[1]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: I can list the libraries that are already imported in the IPython environment for you. These are:

os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses
</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[2]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: Create a 1-line function `checksum` for a string `s`, that multiplies together the ascii 
values of each character in `s` using `reduce`.</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[3]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: ```python
def checksum(s):
  from functools import reduce
  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)
```
</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[4]</summary><ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: Run the code in the IPython environment</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[5]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_call</code>: <ul><li><code>name</code>: run_code</li><li><code>args</code>: <ul><li><b>code</b>: 
def checksum(s):
  from functools import reduce
  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)
</li></ul></li></ul></li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[6]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_response</code>: <ul><li><code>name</code>: run_code</li><li><code>response</code>: <ul><li><b>result</b>: </li></ul></li></ul></li></ul></details></li></ul></details></li></ul></details>

In [217]:
chat.send_message("Use it to get the checksum of the username of this session.", config={'tools': chat._tools})

Press Enter to execute, or enter "n" to skip?
```
username = default_api.get_user()
print(username)

```
 


Looking up username


Press Enter to execute, or enter "n" to skip?
```
def checksum(s):
  from functools import reduce
  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)

username = "Miko"
checksum_value = checksum(username)
print(checksum_value)
```
 


The checksum of the username "Miko" is 96025545.<br />
<details><ul><li><code>candidates</code>: <details open='true'><summary>candidates[0]</summary><ul><li><code>content</code>: <ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: The checksum of the username "Miko" is 96025545.
</li></ul></details></li></ul></li><li><code>avg_logprobs</code>: -0.0005424152172747112</li><li><code>finish_reason</code>: FinishReason.STOP</li></ul></details></li><li><code>model_version</code>: gemini-2.0-flash</li><li><code>usage_metadata</code>: Cached: 0; In: 881; Out: 21; Total: 902</li><li><code>automatic_function_calling_history</code>: <details open='true'><summary>automatic_function_calling_history[0]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: What is already imported in the IPython environment?</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[1]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: I can list the libraries that are already imported in the IPython environment for you. These are:

os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses
</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[2]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: Create a 1-line function `checksum` for a string `s`, that multiplies together the ascii 
values of each character in `s` using `reduce`.</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[3]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: ```python
def checksum(s):
  from functools import reduce
  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)
```
</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[4]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: What is already imported in the IPython environment?</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[5]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: I can list the libraries that are already imported in the IPython environment for you. These are:

os, warnings, time, json, re, math, collections, itertools, functools, dateutil, datetime, string, types, copy, pprint, enum, numbers, decimal, fractions, random, operator, typing, dataclasses
</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[6]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: Create a 1-line function `checksum` for a string `s`, that multiplies together the ascii 
values of each character in `s` using `reduce`.</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[7]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: ```python
def checksum(s):
  from functools import reduce
  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)
```
</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[8]</summary><ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: Run the code in the IPython environment</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[9]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_call</code>: <ul><li><code>name</code>: run_code</li><li><code>args</code>: <ul><li><b>code</b>: 
def checksum(s):
  from functools import reduce
  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)
</li></ul></li></ul></li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[10]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_response</code>: <ul><li><code>name</code>: run_code</li><li><code>response</code>: <ul><li><b>result</b>: </li></ul></li></ul></li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[11]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: OK. I have run the code in the IPython environment. The function `checksum` is now defined.
</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[12]</summary><ul><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: Use it to get the checksum of the username of this session.</li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[13]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_call</code>: <ul><li><code>name</code>: run_code</li><li><code>args</code>: <ul><li><b>code</b>: username = default_api.get_user()
print(username)
</li></ul></li></ul></li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[14]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_response</code>: <ul><li><code>name</code>: run_code</li><li><code>response</code>: <ul><li><b>result</b>: [0;31m---------------------------------------------------------------------------[0m
[0;31mNameError[0m                                 Traceback (most recent call last)
File [0;32m<ipython-input-1-67d2f974c846>:1[0m
[0;32m----> 1[0m username [38;5;241m=[39m [43mdefault_api[49m[38;5;241m.[39mget_user()
[1;32m      2[0m [38;5;28mprint[39m(username)

[0;31mNameError[0m: name 'default_api' is not defined
</li></ul></li></ul></li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[15]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: I apologize, it seems I made a mistake and `default_api` was not available in the IPython environment. I will try again.

</li></ul></details>
<details open='true'><summary>parts[1]</summary><ul><li><code>function_call</code>: <ul><li><code>name</code>: get_user</li><li><code>args</code>: <ul></ul></li></ul></li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[16]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_response</code>: <ul><li><code>name</code>: get_user</li><li><code>response</code>: <ul><li><b>error</b>: Failed to invoke function get_user with converted arguments {} from model returned function call argument {} because of error get_user() missing 1 required positional argument: 'ignored'</li></ul></li></ul></li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[17]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: I apologize again for the error. It seems I need to provide the `ignored` argument.
</li></ul></details>
<details open='true'><summary>parts[1]</summary><ul><li><code>function_call</code>: <ul><li><code>name</code>: get_user</li><li><code>args</code>: <ul><li><b>ignored</b>: </li></ul></li></ul></li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[18]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_response</code>: <ul><li><code>name</code>: get_user</li><li><code>response</code>: <ul><li><b>result</b>: Miko</li></ul></li></ul></li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[19]</summary><ul><li><code>role</code>: model</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>text</code>: Okay, the username is "Miko". Now I can calculate the checksum.
</li></ul></details>
<details open='true'><summary>parts[1]</summary><ul><li><code>function_call</code>: <ul><li><code>name</code>: run_code</li><li><code>args</code>: <ul><li><b>code</b>: def checksum(s):
  from functools import reduce
  return reduce(lambda x, y: x * y, (ord(c) for c in s), 1)

username = "Miko"
checksum_value = checksum(username)
print(checksum_value)</li></ul></li></ul></li></ul></details></li></ul></details>
<details open='true'><summary>automatic_function_calling_history[20]</summary><ul><li><code>role</code>: user</li><li><code>parts</code>: <details open='true'><summary>parts[0]</summary><ul><li><code>function_response</code>: <ul><li><code>name</code>: run_code</li><li><code>response</code>: <ul><li><b>result</b>: 96025545
</li></ul></li></ul></li></ul></details></li></ul></details></li></ul></details>